In [1]:
import pickle
import autograd.numpy as np
from viabel.family import mean_field_gaussian_variational_family
import pandas as pd

import matplotlib.pyplot as plt
import time

In [2]:
data = pd.read_csv("bodyfat.csv").drop("Density", axis=1)
data = data - data.mean()
y = data["Bodyfat"].values
# centre the response to adjust for no intercept
y = y - np.mean(y)

x = data.drop("Bodyfat", axis=1)

N = x.shape[0]
D = x.shape[1]

In [3]:
mc_samples = 100000
M = 100

klvi_probs_mc = np.zeros(shape=(D+1, M))
klvi_probs_sis = np.zeros(shape=(D+1, M))
klvi_probs_psis = np.zeros(shape=(D+1, M))

In [4]:
mf_gaussian_var_family = mean_field_gaussian_variational_family(D+1)

test

In [5]:
with open("./data/klvi_psis_"+str(1)+".pkl", 'rb') as f:
    [btemp, stemp, klvi_samples_temp, klvi_slw_temp, klvi_lw_temp, klvi_khat_temp, klvi_var_param] = pickle.load(f)

In [6]:
klvi_khat_temp

1.0184602207943316

get probs

In [7]:
start = time.time()
for i in range(M):
    start_loop = time.time()
    with open("./data/klvi_psis_"+str(i)+".pkl", 'rb') as f:
        [btemp, stemp, klvi_samples_temp, klvi_slw_temp, klvi_lw_temp, klvi_khat_temp, klvi_var_param] = pickle.load(f)
    
    scaled_weights_sis = np.exp(klvi_lw_temp - max(klvi_lw_temp))
    scaled_weights_psis = np.exp(klvi_slw_temp - max(klvi_slw_temp))
    khat_temp = klvi_khat_temp
    
    for j in range(D+1):
        prob_mc_temp = 0
        prob_sis_temp = 0
        prob_psis_temp = 0
        
        samples_temp = klvi_samples_temp[:][j]
        
        for k in range(mc_samples):
            if (j < D and samples_temp[k] < btemp[0][j]) or (j == D and np.log(samples_temp[k]) < np.log(stemp[0])):
                # unadjusted monte carlo
                prob_mc_temp += 1
                
                # sis
                prob_sis_temp += scaled_weights_sis[k]
                
                # psis
                prob_psis_temp += scaled_weights_psis[k]
                    
        klvi_probs_mc[j, i] = prob_mc_temp / mc_samples
        klvi_probs_sis[j, i] = prob_sis_temp / sum(scaled_weights_sis)
        klvi_probs_psis[j, i] = prob_psis_temp / sum(scaled_weights_psis)
    
    end_loop = time.time()
    print("i = %i, khat = %f, loop time = %f, time elapsed = %f" % (i, khat_temp, end_loop-start_loop, end_loop-start))

end = time.time()
print("Total time taken: %f" % (end-start))

i = 0, khat = 0.935836, loop time = 2.346871, time elapsed = 2.346871
i = 1, khat = 1.018460, loop time = 2.393808, time elapsed = 4.740678
i = 2, khat = 1.030366, loop time = 2.173637, time elapsed = 6.914315
i = 3, khat = 1.013094, loop time = 2.122551, time elapsed = 9.036866
i = 4, khat = 0.961193, loop time = 2.376807, time elapsed = 11.413673
i = 5, khat = 0.908996, loop time = 2.262211, time elapsed = 13.675884
i = 6, khat = 0.965015, loop time = 2.280077, time elapsed = 15.956961
i = 7, khat = 0.972603, loop time = 2.216018, time elapsed = 18.172980
i = 8, khat = 0.957232, loop time = 2.340131, time elapsed = 20.513111
i = 9, khat = 0.927676, loop time = 2.242042, time elapsed = 22.755153
i = 10, khat = 1.075998, loop time = 2.242043, time elapsed = 24.997195
i = 11, khat = 0.998283, loop time = 2.297174, time elapsed = 27.295371
i = 12, khat = 0.926907, loop time = 2.605625, time elapsed = 29.900995
i = 13, khat = 1.058416, loop time = 2.399184, time elapsed = 32.300180
i = 14

In [8]:
np.savetxt("./data/klvi_probs_mc.csv", klvi_probs_mc, delimiter=",")
np.savetxt("./data/klvi_probs_sis.csv", klvi_probs_sis, delimiter=",")
np.savetxt("./data/klvi_probs_psis.csv", klvi_probs_psis, delimiter=",")

In [9]:
chivi_probs_mc = np.zeros(shape=(D+1, M))
chivi_probs_sis = np.zeros(shape=(D+1, M))
chivi_probs_psis = np.zeros(shape=(D+1, M))

In [10]:
start = time.time()
for i in range(M):
    start_loop = time.time()
    with open("./data/chivi_psis_"+str(i)+".pkl", 'rb') as f:
        [btemp, stemp, chivi_samples_temp, chivi_slw_temp, chivi_lw_temp, chivi_khat_temp, chivi_var_param] = pickle.load(f)
    
    scaled_weights_sis = np.exp(chivi_lw_temp - max(chivi_lw_temp))
    scaled_weights_psis = np.exp(chivi_slw_temp - max(chivi_slw_temp))
    khat_temp = chivi_khat_temp
    
    for j in range(D+1):
        prob_mc_temp = 0
        prob_sis_temp = 0
        prob_psis_temp = 0
        
        samples_temp = chivi_samples_temp[:][j]
        
        for k in range(mc_samples):
            if (j < D and samples_temp[k] < btemp[0][j]) or (j == D and np.log(samples_temp[k]) < np.log(stemp[0])):
                # unadjusted monte carlo
                prob_mc_temp += 1
                
                # sis
                prob_sis_temp += scaled_weights_sis[k]
                
                # psis
                prob_psis_temp += scaled_weights_psis[k]
                    
        chivi_probs_mc[j, i] = prob_mc_temp / mc_samples
        chivi_probs_sis[j, i] = prob_sis_temp / sum(scaled_weights_sis)
        chivi_probs_psis[j, i] = prob_psis_temp / sum(scaled_weights_psis)
    
    end_loop = time.time()
    print("i = %i, khat = %f, loop time = %f, time elapsed = %f" % (i, khat_temp, end_loop-start_loop, end_loop-start))

end = time.time()
print("Total time taken: %f" % (end-start))

i = 0, khat = 1.634939, loop time = 2.212591, time elapsed = 2.212591
i = 1, khat = 1.799452, loop time = 2.251967, time elapsed = 4.464558
i = 2, khat = 1.831626, loop time = 2.261194, time elapsed = 6.725751
i = 3, khat = 1.896373, loop time = 2.378145, time elapsed = 9.104897
i = 4, khat = 1.915185, loop time = 2.370159, time elapsed = 11.476057
i = 5, khat = 1.874655, loop time = 2.165453, time elapsed = 13.642510
i = 6, khat = 1.887969, loop time = 2.341375, time elapsed = 15.983885
i = 7, khat = 1.817970, loop time = 2.346140, time elapsed = 18.331026
i = 8, khat = 1.811407, loop time = 2.278446, time elapsed = 20.610473
i = 9, khat = 1.916590, loop time = 2.416481, time elapsed = 23.026954
i = 10, khat = 1.951602, loop time = 2.431214, time elapsed = 25.458168
i = 11, khat = 1.856193, loop time = 2.442819, time elapsed = 27.901989
i = 12, khat = 1.857199, loop time = 2.230381, time elapsed = 30.133370
i = 13, khat = 1.900607, loop time = 2.199003, time elapsed = 32.332373
i = 14

In [11]:
np.savetxt("./data/chivi_probs_mc.csv", chivi_probs_mc, delimiter=",")
np.savetxt("./data/chivi_probs_sis.csv", chivi_probs_sis, delimiter=",")
np.savetxt("./data/chivi_probs_psis.csv", chivi_probs_psis, delimiter=",")